<a href="https://colab.research.google.com/github/YujiSue4Edu/TWMU_2020/blob/master/TWMU_2020_s1_Lecture_02_report_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 課題２
## Pythonによる画像処理  
  以下のいずれかを実行してください  
  
*  PC/スマホに画像を保存してから使いたいとき
  1.  左のフォルダアイコンをクリックして「アップロード」
  をクリック/タップして、画像をアップロードします  
  2.  下セルのコード10行目の「画像名 = 」の右に、引用符でくくった画像ファイル名を記入してください  
  ![代替テキスト](https://user-images.githubusercontent.com/25998652/83730351-e7e9b080-a683-11ea-9d22-932263316543.png)
  
*  ネット上の画像を使いたいとき  
  1.  画像ファイル名を決める  
  下セルのコード10行目の「画像名 = 」の右に、引用符でくくった画像ファイル名を記入してください
  2.  画像のURLを調べる  
  使いたい画像をブラウザで開いて、ブラウザ上部のアドレスをコピーします  
  3.  下のセルの15行目「画像のURL = 」の右側に、引用符でくくって画像のURLをペーストしてください

  ※ 注意：画像のURLは必ず画像本体を示すアドレスでなくてはいけません  
  例えばTwitterなどで画像を開くと、以下のようなアドレスのページが開きますが、これは画像本体を表すURLではありません  
  https://twitter.com/SHARP_JP/status/717912259607994370/photo/1  
  表示される画像の上で「右クリック(Macの場合はControlキー押しながらクリック)」をして、「画像だけを表示」してアドレスをコピーしてください  
  例：https://pbs.twimg.com/media/CfaJZPjUMAAfXas?format=jpg&name=small



In [0]:
# OpenCVなどの導入
import cv2
import numpy as np
import matplotlib.pyplot as plt
import requests
from IPython.display import Image, display_jpeg, display_png, display_pdf


# 画像の名前を指定
画像名 = '*.jpg' # ←画像ファイル名を入れてください

#########　ネット上の画像を使用する場合　######## 
#画像をダウンロード
try:
  画像のURL = 'https;//*' # ←画像のアドレス(URL)を入れてください
  response = requests.get(画像のURL)
  #画像ファイルを保存
  画像ファイル = open(画像名, 'wb')
  画像ファイル.write(response.content)
except Exception as e:
  print(e)

################################################## 

# 画像表示用関数を作る
def showCVImage(mat):
  decoded_bytes = cv2.imencode('.jpg', mat)[1].tobytes()
  display(Image(data=decoded_bytes))

# 画像読み込み
画像 = cv2.imread(画像名)

# 色の抽出
print('元の画像')
showCVImage(画像)
print('\n')

#赤色抽出
下限色 = np.array([0, 0, 127]) # 青, 緑, 赤の順で指定
上限色 = np.array([127, 127, 255])
赤抽出 = cv2.inRange(画像, 下限色, 上限色)
print('赤だけ')
showCVImage(赤抽出)
print('\n')

#青色抽出
下限色 = np.array([127, 0, 0])
上限色 = np.array([255, 127, 127])
青抽出 = cv2.inRange(画像, 下限色, 上限色)
print('青だけ')
showCVImage(青抽出)
print('\n')

#緑色抽出
下限色 = np.array([0, 127, 0])
上限色 = np.array([127, 255, 127])
緑抽出 = cv2.inRange(画像, 下限色, 上限色)
print('緑だけ')
showCVImage(緑抽出)
print('\n')

#特徴検出
グレー画像 = cv2.cvtColor(画像,cv2.COLOR_BGR2GRAY)

# 直線を検出
# 縁（エッジ）検出の閾値
閾値下限 = 75
閾値上限 = 150

# 線分の最低長
最低長 = 200
# 検出
縁検出 = cv2.Canny(グレー画像,閾値下限,閾値上限,apertureSize = 3)
直線リスト = cv2.HoughLines(縁検出,1,np.pi/180,最低長)

# 円形を検出
# 最低中心間距離
最低中心間距離 = 100
# 最低半径
最低半径 = 50
グレー画像 = cv2.medianBlur(グレー画像, 3)
円リスト = cv2.HoughCircles(グレー画像,cv2.HOUGH_GRADIENT,1,最低中心間距離,param1=閾値下限,param2=閾値上限,minRadius=最低半径)
円リスト = np.uint16(np.around(円リスト))

# 検出した個々の直線について
if (直線リスト != None):
  for 線 in 直線リスト:
    for rho,theta in 線:
      # 直線の式ρ＝x･cosθ + y･sinθから線分が通る座標と法線ベクトルを計算
      a = np.cos(theta)
      b = np.sin(theta)
      x0 = a*rho
      y0 = b*rho
      # 線分の端点を算出
      x1 = int(x0 + 200*(-b))
      y1 = int(y0 + 200*(a))
      x2 = int(x0 - 200*(-b))
      y2 = int(y0 - 200*(a))
      # 検出した線を赤く引く
      cv2.line(画像,(x1,y1),(x2,y2),(0,0,255),2)

# 検出した個々の円について
if (0 < len(円リスト[0])):
  for 円 in 円リスト[0]:
    # 検出した円を緑で縁取り
    cv2.circle(画像,(円[0],円[1]),円[2],(0,255,0),2)

print('特徴抽出')
showCVImage(画像)